In [3]:
import pandas as pd
import numpy as np

In [4]:
data=pd.read_csv('freelancer_job_postings.csv')

In [5]:
data

,projectId,job_title,job_description,tags,client_state,client_country,client_average_rating,client_review_count,min_price,max_price,avg_price,currency,rate_type
0,37426471,development and implementation of a federated ...,please bid only if you are ready to do the wor...,"['algorithm', 'java', 'python', 'machine learn...",Heilbronn,Germany,5.0,17,8,30,19.0,EUR,fixed
1,37421546,Data Entry -- 2,Project Title: Data Entry - Data Analysis in E...,"['excel', 'statistical analysis', 'statistics'...",Nagpur,India,0.0,0,750,1250,1000.0,INR,hourly
2,37400492,Data Scrap,I am looking for a freelancer who can help me ...,"['web scraping', 'data mining', 'data entry', ...",Eaubonne,France,5.0,1,30,250,140.0,EUR,fixed
3,37404568,Big Data Project,Store Sales Data Analysis: A Data Engineering ...,"['big data sales', 'data science', 'data minin...",Mundra,India,5.0,2,5000,5500,5250.0,INR,fixed
4,37397423,Virtual Assistant / Research Assistant,Job Description: I am seeking a Virtual Assist...,"['data entry', 'virtual assistant', 'web searc...",NaN,United States,0.0,0,5,15,10.0,USD,hourly
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9188,31183091,Computer Vision / Feature Extraction from Aeri...,Looking for an experienced computer vision / f...,"['python', 'deep learning', 'computer vision',...",Buenos Aires,Argentina,5.0,12,30,250,140.0,USD,fixed
9189,31184739,Database and software analysis,I need a freelancer who's proficient in micros...,['data processing'],NaN,United Kingdom,0.0,0,250,750,500.0,USD,fixed
9190,31185401,Business Analyst (Strategic Projects) -- 2,Role Purpose The role’s purpose is to superch...,"['python', 'business analysis', 'financial ana...",Sydney,Australia,5.0,39,15,25,20.0,AUD,hourly
9191,31189015,AI developer needed,We are an italian startup seeking for a machin...,"['data analysis', 'machine learning (ml)', 'py...",Modena,Italy,0.0,0,3000,5000,4000.0,EUR,fixed


In [6]:
# Example fixed exchange rates (as of Aug 2025 approx — update with latest)
exchange_rates = {
    "USD": 1.0,
    "EUR": 1.10,   # 1 EUR = 1.10 USD
    "INR": 0.012,  # 1 INR = 0.012 USD
    "SGD": 0.74,   # 1 SGD = 0.74 USD
    "GBP": 1.27,   # 1 GBP = 1.27 USD
    "AUD": 0.67,   # 1 AUD = 0.67 USD
    "HKD": 0.13,   # 1 HKD = 0.13 USD
    "CAD": 0.73,   # 1 CAD = 0.73 USD
    "NZD": 0.61    # 1 NZD = 0.61 USD
}

# Function to convert given price column to USD
def convert_to_usd(row, column):
    rate = exchange_rates.get(row["currency"], 1.0)
    return row[column] * rate

# Convert min_price, max_price, avg_price to USD
for col in ["min_price", "max_price", "avg_price"]:
    data[f"{col}_usd"] = data.apply(lambda row: convert_to_usd(row, col), axis=1)

# Update currency column to USD
data["currency"] = "USD"

# Preview
print(data[["min_price_usd", "max_price_usd", "avg_price_usd"]].head())

   min_price_usd  max_price_usd  avg_price_usd
0            8.8           33.0           20.9
1            9.0           15.0           12.0
2           33.0          275.0          154.0
3           60.0           66.0           63.0
4            5.0           15.0           10.0


In [7]:
data.drop(['currency','projectId','client_state'], axis=1, inplace=True)

In [8]:
data.drop(columns=["min_price", "max_price", "avg_price"], inplace=True)

In [9]:
data.drop(['client_average_rating','client_review_count'], axis=1, inplace=True)

In [10]:
import numpy as np

# Replace empty strings, "NA", "nan", whitespace with NaN
data['client_country'] = data['client_country'].replace(
    ["", " ", "NA", "NAN", "nan"], np.nan
)

# Drop rows where client_country is NaN
data = data.dropna(subset=['client_country'])

In [11]:
import pandas as pd

# One-hot encode rate_type
data = pd.get_dummies(data, columns=['rate_type'], prefix='rate')

# Check result


In [12]:
# Convert boolean columns to integers
data['rate_fixed'] = data['rate_fixed'].astype(int)
data['rate_hourly'] = data['rate_hourly'].astype(int)

print(data[['rate_fixed','rate_hourly']].head())


   rate_fixed  rate_hourly
0           1            0
1           0            1
2           1            0
3           1            0
4           0            1


In [13]:
import pandas as pd
import ast

# Load your dataset


# Function to clean and convert tags list → string
def process_tags(tag_str):
    try:
        # Convert string representation of list into actual list
        tag_list = ast.literal_eval(tag_str)
        
        # Join list elements into a comma-separated string
        if isinstance(tag_list, list):
            return ", ".join([str(t).strip() for t in tag_list])
        else:
            return str(tag_str)
    except:
        return str(tag_str)  # fallback if not list-like

# Apply function to your tags column
data["tags_clean"] = data["tags"].apply(process_tags)

# Check result
print(data[["tags", "tags_clean"]].head())

                                                tags  \
0  ['algorithm', 'java', 'python', 'machine learn...   
1  ['excel', 'statistical analysis', 'statistics'...   
2  ['web scraping', 'data mining', 'data entry', ...   
3  ['big data sales', 'data science', 'data minin...   
4  ['data entry', 'virtual assistant', 'web searc...   

                                          tags_clean  
0  algorithm, java, python, machine learning (ml)...  
1  excel, statistical analysis, statistics, spss ...  
2  web scraping, data mining, data entry, excel, ...  
3  big data sales, data science, data mining, sta...  
4  data entry, virtual assistant, web search, exc...  


In [14]:
data.drop(['tags'], axis=1, inplace=True)

In [15]:
df=data

In [17]:
import pandas as pd
from sentence_transformers import SentenceTransformer


d:\AI\Mastery series\Projects Aug-Sep 2025\freelancecontracts\freelancer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
import torch
print("Torch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("GPU Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

Torch version: 2.8.0+cpu
CUDA version: None
GPU Available: False


In [22]:
df["text_for_embedding"] = (
    df["job_title"].astype(str) + " | " +
    df["job_description"].astype(str) + " | " +
    df["tags_clean"].astype(str)
)

# Load embedding model (you can also try 'all-MiniLM-L6-v2' for speed)
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Generate embeddings
embeddings = model.encode(df["text_for_embedding"].tolist(), show_progress_bar=True)

# Save embeddings into dataframe
df["embeddings"] = embeddings.tolist()

# Save processed file
df.to_pickle("dataset_with_embeddings.pkl")  # preserves numpy arrays


Batches: 100%|██████████| 288/288 [26:01<00:00,  5.42s/it]    


In [24]:
df.drop(['tags_clean'],axis=1, inplace=True)

In [25]:
# Select only input + output columns
train_df = df[['embeddings', 'client_country', 
               'rate_fixed', 'rate_hourly', 
               'min_price_usd', 'max_price_usd']].copy()

print(train_df.head())

                                          embeddings client_country  \
0  [-0.0010373883415013552, 0.02592027746140957, ...        Germany   
1  [0.013640339486300945, 0.07931948453187943, -0...          India   
2  [0.05617599934339523, 0.07921624183654785, -0....         France   
3  [-0.010524313896894455, 0.04207714647054672, -...          India   
4  [0.05351153761148453, 0.013150510378181934, -0...  United States   

   rate_fixed  rate_hourly  min_price_usd  max_price_usd  
0           1            0            8.8           33.0  
1           0            1            9.0           15.0  
2           1            0           33.0          275.0  
3           1            0           60.0           66.0  
4           0            1            5.0           15.0  


In [26]:
train_df

,embeddings,client_country,rate_fixed,rate_hourly,min_price_usd,max_price_usd
0,"[-0.0010373883415013552, 0.02592027746140957, ...",Germany,1,0,8.80,33.00
1,"[0.013640339486300945, 0.07931948453187943, -0...",India,0,1,9.00,15.00
2,"[0.05617599934339523, 0.07921624183654785, -0....",France,1,0,33.00,275.00
3,"[-0.010524313896894455, 0.04207714647054672, -...",India,1,0,60.00,66.00
4,"[0.05351153761148453, 0.013150510378181934, -0...",United States,0,1,5.00,15.00
...,...,...,...,...,...,...
9188,"[0.02490326203405857, 0.0824948102235794, -0.0...",Argentina,1,0,30.00,250.00
9189,"[0.026110423728823662, 0.0825563296675682, -0....",United Kingdom,1,0,250.00,750.00
9190,"[0.037851691246032715, 0.04596949368715286, -0...",Australia,0,1,10.05,16.75
9191,"[-0.018802417442202568, 0.06984639167785645, -...",Italy,1,0,3300.00,5500.00


In [27]:
# Create a boolean mask for fixed-price projects
is_fixed_mask = train_df['rate_fixed'] == 1

# Split into two dataframes
fixed_df = train_df[is_fixed_mask]
hourly_df = train_df[~is_fixed_mask]

In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [31]:
import numpy as np
import pandas as pd
import tensorflow as tf
import joblib
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error

# --- Configuration ---
# Make sure the pickle file with your embeddings is in the same folder
EMBEDDINGS_FILENAME = 'dataset_with_embeddings.pkl'

# --- 1. Load Pre-processed Data ---
print("--- 1. Loading Data with Embeddings ---")
try:
    train_df = pd.read_pickle(EMBEDDINGS_FILENAME)
    print(f"Successfully loaded '{EMBEDDINGS_FILENAME}'.")
except FileNotFoundError:
    print(f"FATAL ERROR: The file '{EMBEDDINGS_FILENAME}' was not found.")
    print("Please run your embedding generation script first.")
    exit()

# --- 2. Initial Preprocessing ---
print("\n--- 2. Preprocessing Data for Training ---")
# Encode country names into integer IDs
country_encoder = LabelEncoder()
train_df['country_encoded'] = country_encoder.fit_transform(train_df['client_country'])
num_countries = len(country_encoder.classes_)

# Save the fitted LabelEncoder for use in the Flask app
joblib.dump(country_encoder, 'country_encoder.joblib')
print("Country encoder created and saved to 'country_encoder.joblib'.")

# Split into two separate dataframes
is_fixed_mask = train_df['rate_fixed'] == 1
fixed_df = train_df[is_fixed_mask].copy()
hourly_df = train_df[~is_fixed_mask].copy()
print(f"Split data into {len(fixed_df)} fixed-price and {len(hourly_df)} hourly-rate projects.\n")


# --- 3. Identify and Remove Outliers from Each Dataset ---
def remove_outliers(df, column_name, df_name):
    """Identifies and removes outliers based on the IQR method."""
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    
    num_outliers = df[df[column_name] > upper_bound].shape[0]
    
    # Filter the dataframe to keep only non-outliers
    df_cleaned = df[df[column_name] <= upper_bound].copy()
    
    print(f"For {df_name}:")
    print(f"  Removed {num_outliers} high-budget outliers (prices > ${upper_bound:,.2f}).")
    print(f"  Remaining projects for training/testing: {len(df_cleaned)}\n")
    return df_cleaned

fixed_df_cleaned = remove_outliers(fixed_df, 'max_price_usd', 'Fixed-Price Projects')
hourly_df_cleaned = remove_outliers(hourly_df, 'max_price_usd', 'Hourly-Rate Projects')


# --- 4. Define Model Architecture ---
embedding_dim = np.array(train_df['embeddings'].iloc[0]).shape[0]

def build_regressor(embedding_dim, num_countries):
    """Builds a robust regression model."""
    text_in = layers.Input(shape=(embedding_dim,), name="text_embeddings")
    country_in = layers.Input(shape=(1,), name="country")
    
    country_emb = layers.Embedding(input_dim=num_countries + 1, output_dim=16)(country_in)
    country_emb = layers.Flatten()(country_emb)
    
    concatenated = layers.Concatenate()([text_in, country_emb])
    
    x = layers.Dense(128, activation='relu')(concatenated)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(2, activation='linear', name='price_range')(x)
    
    model = Model(inputs=[text_in, country_in], outputs=output)
    return model

# --- 5. Train and Evaluate a Model for Each Dataset ---
def train_and_evaluate_model(df, df_name, save_filename):
    """Splits data, trains a model, and evaluates it."""
    print(f"--- Processing Model for {df_name} ---")
    
    # Split Data
    train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
    print(f"Split cleaned data into {len(train_data)} for training and {len(test_data)} for testing.")
    
    # Prepare Training Data
    y_log_train = np.log1p(train_data[['min_price_usd', 'max_price_usd']].values)
    X_train = {
        "text_embeddings": np.array(train_data['embeddings'].tolist()),
        "country": train_data['country_encoded'].values
    }
    
    # Prepare Test/Validation Data
    y_log_test = np.log1p(test_data[['min_price_usd', 'max_price_usd']].values)
    X_test = {
        "text_embeddings": np.array(test_data['embeddings'].tolist()),
        "country": test_data['country_encoded'].values
    }
    
    # Build and Compile
    regressor = build_regressor(embedding_dim, num_countries)
    regressor.compile(optimizer=Adam(learning_rate=0.001), loss='huber')
    
    # Train
    stop_early = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    regressor.fit(X_train, y_log_train,
                  validation_data=(X_test, y_log_test),
                  epochs=100, batch_size=32, callbacks=[stop_early], verbose=1)
    print("Training complete.")
    
    # Evaluate
    y_test_actual = test_data[['min_price_usd', 'max_price_usd']].values
    log_preds = regressor.predict(X_test)
    final_dollar_preds = np.expm1(log_preds)
    
    mae = mean_absolute_error(y_test_actual, final_dollar_preds)
    rmse = np.sqrt(mean_squared_error(y_test_actual, final_dollar_preds))
    
    print(f"\n--- Final Metrics for {df_name} ---")
    print(f"Test MAE: ${mae:,.2f}")
    print(f"Test RMSE: ${rmse:,.2f}\n")

    # Save the final model in the .h5 format
    regressor.save(save_filename)
    print(f"--- Model saved to {save_filename} ---\n")

# Run the process for both fixed-price and hourly-rate projects
train_and_evaluate_model(fixed_df_cleaned, "Fixed-Price Projects", "fixed_price_model.h5")
train_and_evaluate_model(hourly_df_cleaned, "Hourly-Rate Projects", "hourly_rate_model.h5")

print("--- ML Training Pipeline Complete ---")

--- 1. Loading Data with Embeddings ---
Successfully loaded 'dataset_with_embeddings.pkl'.

--- 2. Preprocessing Data for Training ---
Country encoder created and saved to 'country_encoder.joblib'.
Split data into 7321 fixed-price and 1871 hourly-rate projects.

For Fixed-Price Projects:
  Removed 1322 high-budget outliers (prices > $748.75).
  Remaining projects for training/testing: 5999

For Hourly-Rate Projects:
  Removed 149 high-budget outliers (prices > $49.00).
  Remaining projects for training/testing: 1722

--- Processing Model for Fixed-Price Projects ---
Split cleaned data into 4799 for training and 1200 for testing.
Epoch 1/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.8931 - val_loss: 0.4451
Epoch 2/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4506 - val_loss: 0.4001
Epoch 3/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4104 - val_loss: 0.4080
Epoch 4/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3996 - val_loss: 0.3926
Epoch 5/100
150


--- Final Metrics for Fixed-Price Projects ---
Test MAE: $65.05
Test RMSE: $109.18

--- Model saved to fixed_price_model.h5 ---

--- Processing Model for Hourly-Rate Projects ---
Split cleaned data into 1377 for training and 345 for testing.
Epoch 1/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.6643 - val_loss: 0.2747
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2648 - val_loss: 0.2433
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2160 - val_loss: 0.2243
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1900 - val_loss: 0.2172
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1694 - val_loss: 0.2072
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1720 - val_loss: 0.2145
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1671 - val_loss: 0.2082
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1597 - val_loss: 0.2023
Epoch 9/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1620 - val_loss:


--- Final Metrics for Hourly-Rate Projects ---
Test MAE: $5.64
Test RMSE: $7.46

--- Model saved to hourly_rate_model.h5 ---

--- ML Training Pipeline Complete ---


NameError: name 'fixed_model' is not defined

In [5]:
import keras

# Load old H5 models (using tf.keras inside keras 3 is fine)
old_fixed = keras.models.load_model("fixed_price_model.h5", compile=False)
old_hourly = keras.models.load_model("hourly_rate_model.h5", compile=False)

# Save them in new Keras 3 format
old_fixed.save("fixed_price_model_new.keras", save_format="keras")
old_hourly.save("hourly_rate_model_new.keras", save_format="keras")

print("✅ Models converted successfully to Keras 3 format")


✅ Models converted successfully to Keras 3 format
